In [2]:
print("Hello World")

Hello World


In [3]:
!pip install langchain==0.1.14 langchain-experimental==0.0.56 langchain-community==0.0.31 faiss-cpu==1.8.0 pdfplumber==0.11.0 gradio==4.25.0 sentence-transformers

In [3]:
from langchain_community.document_loaders import PDFPlumberLoader
loader = PDFPlumberLoader("./pdf/CHQAutomobile.pdf")
docs = loader.load()

# Check the number of pages
print("Number of pages in the PDF:",len(docs))

# Load the random page content
docs[5].page_content

Number of pages in the PDF: 6


'Effect of Heating Rate on Microstructural Developments in Cold Heading Quality Steel used for Automotive Applications\n[21] Caballero, F.G., and Capdevila,C., “Modelling of Kinetics [23] Huang, J., Poole, W.,and Militzer, M., “Austenite\nFormation During Intercritical Annealing”, Metallurgical\nof Austenite Formation in Steels with Different Initial\nand Materials Transactions-A, Volume 35, No. 11, pp.\nMicrostructures”, International Scientific Invention 3363–3375, November, 2004.\nJournals, pp. 1093-1102, 2001.\n[24] Humphreys, F.J., and Hatherly, M., “Recrystallization\nand Related Annealing Phenomena”, 1st Edition,\n[22] Oliveiraa, F.L.G., and Andrade, S., “Kinetics of Austenite\nPergamon, pp. 127–71, New York, 1995.\nFormation During Continuous Heating in a Low Carbon\n[25] Mecozzi, J., Sietsma, S.,and van der Zwaag, M.,\nSteel”, Journal of Materials Characterization, Volume\n“Austenite Formation and Decomposition”, ISS and TMS,\n58, No. 3, pp. 256-261, March, 2007. pp. 353-66, W

In [10]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain.embeddings import HuggingFaceEmbeddings

text_splitter = SemanticChunker(HuggingFaceEmbeddings())
documents = text_splitter.split_documents(docs)

In [12]:
print("Number of chunks created: ", len(documents))
# Output 
"""
Number of chunks created:  23
"""

# Printing first few chunks
for i in range(len(documents)):
    print()
    print(f"CHUNK : {i+1}")
    print(documents[i].page_content)

Number of chunks created:  15

CHUNK : 1
Mehran University Research Journal of Engineering & Technology
Vol.

CHUNK : 2
37, No. 3, 461-466, July 2018
p-ISSN: 0254-7821, e-ISSN: 2413-7219
DOI: 10.22581/muet1982.1803.02
Effect of Heating Rate on Microstructural Developments in Cold
Heading Quality Steel used for Automotive Applications
SHAHID HUSSAIN ABRO*, ALI DAD CHANDIO*, AND UMAIR AFTAB**
RECEIVED ON 18.01.2017 ACCEPTED ON 29.05.2017
ABSTRACT
CHQ (Cold Heading Quality) steels are assumed to be non-heat treatable so strengthened by cold forming
which is a quick and mass production makes it low cost manufacturing process. In this study,
microstructure and phase transformation in steels is a fundamental aspect of governing the properties. A large number of studies are dedicated on the subject phase transformations owing to availability of
several commercial steel grades. Due to several grades of steels commercially available shows differently
behavior so it is hard to conclude that phas

In [14]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# Instantiate the embedding model
embedder = HuggingFaceEmbeddings()

# Create the vector store 
vector = FAISS.from_documents(documents, embedder)

In [16]:
retriever = vector.as_retriever(search_type="similarity", search_kwargs={"k": 3})
retrieved_docs = retriever.invoke("How does plant respond to disease?")


In [18]:
from langchain_community.llms import Ollama

# Define llm
llm = Ollama(model="llama3.2:1b")

In [20]:
from langchain.chains import RetrievalQA
from langchain.chains.llm import LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.prompts import PromptTemplate

prompt = """
1. Use the following pieces of context to answer the question at the end.
2. If you don't know the answer, just say that "I don't know" but don't make up an answer on your own.\n
3. Keep the answer crisp and limited to 3,4 sentences.

Context: {context}

Question: {question}

Helpful Answer:"""

QA_CHAIN_PROMPT = PromptTemplate.from_template(prompt) 

llm_chain = LLMChain(
                  llm=llm, 
                  prompt=QA_CHAIN_PROMPT, 
                  callbacks=None, 
                  verbose=True)

document_prompt = PromptTemplate(
    input_variables=["page_content", "source"],
    template="Context:\ncontent:{page_content}\nsource:{source}",
)

combine_documents_chain = StuffDocumentsChain(
                  llm_chain=llm_chain,
                  document_variable_name="context",
                  document_prompt=document_prompt,
                  callbacks=None,
              )

qa = RetrievalQA(
                  combine_documents_chain=combine_documents_chain,
                  verbose=True,
                  retriever=retriever,
                  return_source_documents=True,
              )


In [22]:
print(qa("Who wrote the paper?")["result"])

C:\Users\sapta\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new RetrievalQA chain...


> Entering new LLMChain chain...
Prompt after formatting:

1. Use the following pieces of context to answer the question at the end.
2. If you don't know the answer, just say that "I don't know" but don't make up an answer on your own.

3. Keep the answer crisp and limited to 3,4 sentences.

Context: Context:
content:647-654, August, 2012. Examination, pp.
source:./pdf/CHQAutomobile.pdf

Context:
content:1986. A, Volume 25, No. 4, pp. 763-773 , 1994. [6] Savran, V.I., “Microstructural Features of Austenite [16] Caballero, F.G., Bhadeshia, H.K.D.H., Mawella, K.J.A.,
Formation in C35 and C45 Alloys, Metallurgical and and Jones, D.G., “Very Strong Low Temperature Bainite”,
Materials Transactions-A, pp. 946-955, 2007. Materials Science and Technology, pp. 279. 2002
[7] Huang, J., “Austenite Formation During Intercritical [17] Karmakar, A., Sivaprasad, S., Kundu, S., and Chakrabarti,
Annealing”,Metallurgical and Materials Transactions- D., “Tensile Be

In [30]:
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain.chains.llm import LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains import RetrievalQA
import gradio as gr

# Load the PDF
loader = PDFPlumberLoader("./pdf/CHQAutomobile.pdf")
docs = loader.load()

# Split into chunks
text_splitter = SemanticChunker(HuggingFaceEmbeddings())
documents = text_splitter.split_documents(docs)


# Instantiate the embedding model
embedder = HuggingFaceEmbeddings()

# Create the vector store and fill it with embeddings
vector = FAISS.from_documents(documents, embedder)
retriever = vector.as_retriever(search_type="similarity", search_kwargs={"k": 3})

# Define llm
llm = Ollama(model="llama3.2:1b")

# Define the prompt
prompt = """
1. Use the following pieces of context to answer the question at the end.
2. If you don't know the answer, just say that "I don't know" but don't make up an answer on your own.\n
3. Keep the answer crisp and limited to 3,4 sentences.

Context: {context}

Question: {question}

Helpful Answer:"""

QA_CHAIN_PROMPT = PromptTemplate.from_template(prompt) 

llm_chain = LLMChain(
                  llm=llm, 
                  prompt=QA_CHAIN_PROMPT, 
                  callbacks=None, 
                  verbose=True)

document_prompt = PromptTemplate(
    input_variables=["page_content", "source"],
    template="Context:\ncontent:{page_content}\nsource:{source}",
)

combine_documents_chain = StuffDocumentsChain(
                  llm_chain=llm_chain,
                  document_variable_name="context",
                  document_prompt=document_prompt,
                  callbacks=None)
              
qa = RetrievalQA(
                  combine_documents_chain=combine_documents_chain,
                  verbose=True,
                  retriever=retriever,
                  return_source_documents=True)

def respond(question,history):
    return qa(question)["result"]


gr.ChatInterface(
    respond,
    chatbot=gr.Chatbot(height=500),
    textbox=gr.Textbox(placeholder="Ask me question related to the paper", container=False, scale=7),
    title="Research Paper Chatbot",
    examples=["Summarize the paper", "Who wrote the paper"],
    cache_examples=True,
    retry_btn=None,

).launch(share = True, server_port=6969)

KeyboardInterrupt: 